<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Keras_WithDeeplizard/blob/master/3_Data_Processing_For_Neural_Network_Training_With_TensorFlow's_Keras_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keras With TensorFlow - Data Processing For Neural Network Training
我们将演示如何处理数值数据，这些数值数据稍后将用于训练我们的第一个人工神经网络。

## 1.样本和标签
在监督学习任务中训练任何神经网络，我们首先需要一个样本数据集和这些样本对应的标签。

当提到样本时，我们仅指的是基础数据集，其中该集中的每个单独的项目或数据点都称为样本。 标签是样本的相应标签。

例如，如果我们训练一个模型来对一个媒体来源的标题进行情感分析，那么每个样本标题对应的标签可以是正面的，也可以是负面的。

如果我们用猫和狗的图像训练一个模型，那么每个图像的标签要么是猫，要么是狗。

**注意：**在深度学习中，样本也通常被称为输入数据或输入，标签也通常被称为目标数据或目标。

## 2.期望的数据格式
在准备数据时，我们首先需要理解数据为实现我们心中的最终目标所需要的格式。在我们的例子中，我们希望我们的数据采用一种可以传递给神经网络模型的格式。

我们将在下一集中构建的第一个模型将是TensorFlow中集成的Keras API的顺序模型。 我们将在以后的文章中讨论这种类型的模型的详细信息，但就目前而言，我们只需要了解顺序模型所需的数据类型即可。

序列模型在训练期间接收数据，这发生在模型调用fit()函数时。因此，我们需要检查这个函数所期望的数据类型。

根据fit()函数的[API](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#fit)，输入数据x需要是以下数据类型之一:

* A Numpy array (or array-like), or a list of arrays (in case the model has multiple inputs).
* A TensorFlow tensor, or a list of tensors (in case the model has multiple inputs).
* A dict mapping input names to the corresponding array/tensors, if the model has named inputs.
* A tf.data dataset. Should return a tuple of either (inputs, targets) or (inputs, targets, sample_weights).
* A generator or keras.utils.Sequence returning (inputs, targets) or (inputs, targets, sample_weights).

因此，当我们聚合数据时，我们需要确保它包含在上述类型的数据结构中。数据对应的标签y应该采用类似的格式。

与输入数据x一样，对应的标签数据y也可以是Numpy数组或TensorFlow张量。注意，y应该与x一致。我们不能有Numpy样本和张量标签，反之亦然。

**注意：**如果x是dataset, generator, or keras.utils.Sequence instance，则不应该指定y(因为标签将从x获得)。我们将在后面的课程中看到这个例子。

除了格式化数据以使其符合模型要求的格式外，格式化或处理数据的另一个原因是对数据进行转换，以使其可以更轻松，更快或更有效地用于网络。我们也可以通过数据标准化或标准化技术来做到这一点。

## 3. 数据处理
深度学习的数据处理将根据我们处理的数据类型和我们将使用网络完成的任务类型有很大差异。

我们将从使用简单的数值数据集的非常简单的分类任务开始。 在课程的后面，我们将处理其他类型的数据和其他任务。

### 3.1 导入库

In [ ]:
import numpy as np
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_labels = []
train_samples = []


### 3.2 创建数据
对于此简单任务，我们将创建自己的示例数据集。

为了获得这些数据，我们假设在一项临床试验中对13至100岁的个体进行了实验药物测试。 该试验有2100名参与者。 一半的参与者年龄在65岁以下，另一半则是65岁以上。

试验表明，大约95%的65岁或以上的患者经历了药物的副作用，而大约95%的65岁以下的患者没有副作用，总体上表明老年人更有可能经历副作用。

最终，我们想建立一个模型来告诉我们患者是否会仅仅根据患者的年龄来经历副作用。模型的判断将基于训练数据。

**注意：**由于数据的简单性以及从中得出的结论，神经网络可能有些过头了，但理解这一点只是首先介绍如何使用数据进行深度学习，之后，我们将使用更高级的数据集。



In [ ]:
# 生成数据
for i in range(50):
    # 大约5%的年轻人确实经历过副作用
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(1)

    # 大约5%的老年人没有经历过副作用
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(0)

for i in range(1000):
    # 大约95%的年轻人没有经历过副作用
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(0)

    # 大约95%的老年人确实经历过副作用
    random_older = randint(65,100)
    train_samples.append(random_older)
    train_labels.append(1)

这段代码创建了2100个样本，并将个体的年龄存储在train样本列表中，并将个体是否经历副作用存储在train标签列表中。
这就是train_samples数据和train_labels的样子。



```
for i in train_samples:
    print(i)
for i in train_labels:
    print(i)
```

0表示个体没有经历过副作用，1表示个体确实经历过副作用。

### 3.3 数据处理
根据fit()函数所期望的，我们现在将两个列表转换为numpy数组，然后打乱数组，以删除创建过程中强加给数据的任何顺序。

In [ ]:
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)
train_labels,train_samples = shuffle(train_labels,train_samples)
print(train_samples.shape)

(6300,)


通过这种形式，我们现在可以将数据传递到模型，因为它现在是必需的格式，但是在此之前，我们首先将数据缩小到0到1的范围。

我们将使用scikit-learn的MinMaxScaler类来将所有数据从13到100的范围缩放到从0到1的范围。

In [ ]:
# 通过将每个特征缩放到给定的范围来转换特征。
scaler = MinMaxScaler(feature_range=(0,1))

scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))
# 我们只是根据fit_transform（）函数默认情况下不接受一维数据的情况，将数据重塑为2D。
print(train_samples.reshape(-1,1).shape)

(6300, 1)


为了进一步理解我们为什么要以这种方式缩小数据规模，请查看深度学习基础课程中关于批处理规范化的前半节，我们在这一节中讨论了标准化和规范化技术。

现在数据已经缩放，让s遍历缩放后的数据，看看它现在是什么样子。



```
for i in scaled_train_samples:
    print(i)
```

正如预期的那样，所有的数据都被转换为0到1之间的数字。

现在，我们已经生成了一些示例原始数据，将其放入我们的模型需要的numpy格式中，并将其缩放到从0到1的范围。

在下一集中，我们将使用这些数据来训练一个神经网络，看看我们可以得到什么样的结果